# Fantasy Squads

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append('..')
from wyscout.competitions import get_competitions, get_competition_players, get_all_competition_players
from wyscout.match import get_competition_matches
from wyscout.team import get_player_stats
from prem_fl import get_details


In [3]:
ENGLAND = "XEN"
PREMIER_LEAGUE = 364
SEASON_2022 = 188095
STD_SEASON = 4426845

In [7]:
details = get_details()
fl_players = details["elements"]
fl_positions_map = {p["id"]: p for p in details["element_types"]}
fl_player_map = {p["code"]: p for p in details["elements"]}
for k in fl_player_map.keys():
    fl_player_map[k]["position"] = fl_positions_map[fl_player_map[k]["element_type"]]["plural_name"]

In [8]:
wy_players = get_all_competition_players(PREMIER_LEAGUE)

### Simple player map

In [9]:
matched = 0
for i, wp in enumerate(wy_players):
    found = False
    for j, flp in enumerate(fl_players):
        if wp["firstName"].strip() == flp["first_name"].strip() and wp["lastName"].strip() == flp["second_name"].strip():
            matched += 1
            wy_players[i]["flId"] = flp["code"]
            found = True
            break
    if found: 
        continue
print(matched)

429


In [10]:
ws_mapped_players = [p for p in wy_players if "flId" in p]
ws_unmapped_players = [p for p in wy_players if "flId" not in p]

In [11]:
data = get_player_stats(ws_mapped_players[0]["wyId"], SEASON_2022, PREMIER_LEAGUE, STD_SEASON)

In [13]:
data = []
cols = []


for p in ws_mapped_players:
    fl_p = fl_player_map[p["flId"]]
    wy_p = get_player_stats(p["wyId"], SEASON_2022, PREMIER_LEAGUE, STD_SEASON)
    if len(cols) == 0:
        cols.extend(list(wy_p["total"].keys()))
    player_data = [p["wyId"], p["shortName"]]
    player_data.extend([wy_p["total"][col] for col in cols])
    player_data.extend([
        fl_p["position"],
        fl_p["now_cost"],
        fl_p["total_points"]
    ])
    data.append(player_data)

cols.extend(["position", "cost", "points"])

cols = ["playerId", "name"] + cols

In [14]:
lines = [(",".join(cols))]
for d in data:
    ds = [str(di) for di in d]
    lines.extend([",".join(ds)])
    
f = open("../ludwig/player_data.csv", "a")
for l in lines:
    f.write(f"{l}\n")
f.close()

In [ ]:
for col in cols:
    print(f"    - name: {col}")
    print("      type: number")

# Run training and get predictions

```
ludwig train --dataset player_data.csv   --config config.yaml
ludwig predict --dataset player_data.csv  --model_path=results/experiment_run/model
```

In [15]:
import csv

predictions_file = "../ludwig/results/cost_predictions.csv"

with open(predictions_file, newline='') as f:
    reader = csv.reader(f)
    predicted_costs = list(reader)


In [16]:
import pandas as pd
df = pd.DataFrame(data, columns=cols)
df["predictedCost"] = [p[0] for p in predicted_costs]
df["predictedCost"] = pd.to_numeric(df["predictedCost"])
df["cost"] = pd.to_numeric(df["cost"])
df["predictedCostDiff"] = df["cost"] - df["predictedCost"]

In [17]:
df.sort_values("predictedCostDiff").head(20)

,playerId,name,matches,matchesInStart,matchesSubstituted,matchesComingOff,minutesOnField,minutesTagged,goals,assists,...,newDefensiveDuelsWon,newOffensiveDuelsWon,newSuccessfulDribbles,lateralPasses,successfulLateralPasses,position,cost,points,predictedCost,predictedCostDiff
383,470834,N. Williams,15,12,3,3,1181,1181,0,0,...,70,56,36,143,101,Defenders,40,30,104.614960,-64.614960
257,248276,Daniel Podence,15,12,4,3,1137,1137,2,0,...,26,39,25,110,85,Midfielders,53,44,104.210899,-51.210899
171,134717,L. Trossard,14,14,9,0,1251,1251,7,2,...,22,46,32,141,101,Midfielders,71,80,96.617485,-25.617485
80,550459,J. Álvarez,12,3,2,9,416,416,3,0,...,12,9,5,36,29,Forwards,60,32,85.186813,-25.186813
350,412968,M. Senesi,12,11,3,1,1032,1032,0,0,...,56,8,3,150,144,Defenders,45,22,64.124092,-19.124092
201,9380,J. Stones,9,8,1,1,804,804,0,2,...,24,7,4,318,309,Defenders,54,32,72.953323,-18.953323
88,7879,T. Walcott,3,1,1,2,129,129,0,0,...,4,0,0,9,3,Midfielders,48,3,66.367973,-18.367973
372,448645,M. Gibbs-White,15,14,4,1,1327,1327,2,2,...,36,53,27,104,81,Midfielders,55,55,72.845245,-17.845245
422,532949,D. McNeil,15,8,6,7,812,812,2,0,...,44,26,15,63,49,Midfielders,52,34,69.691032,-17.691032
134,9624,T. Cairney,15,1,1,14,427,427,0,1,...,14,14,7,104,92,Midfielders,47,19,64.359009,-17.359009


In [65]:
df.sort_values("points", ascending=False).head(20)

,playerId,name,matches,matchesInStart,matchesSubstituted,matchesComingOff,minutesOnField,minutesTagged,goals,assists,...,newDefensiveDuelsWon,newOffensiveDuelsWon,newSuccessfulDribbles,lateralPasses,successfulLateralPasses,position,cost,points,predictedCost,predictedCostDiff
176,8717,H. Kane,15,15,3,0,1447,1447,12,0,...,21,60,38,104,71,Forwards,116,102,118.392479,-2.392479
139,8945,K. Trippier,15,15,3,0,1471,1471,1,3,...,61,17,8,226,192,Defenders,59,96,51.589424,7.410576
127,38021,K. De Bruyne,14,13,5,1,1232,1232,3,9,...,19,50,27,279,222,Midfielders,126,90,125.769936,0.230064
254,243886,I. Toney,14,14,0,0,1392,1392,10,2,...,25,34,10,64,46,Forwards,74,83,77.714722,-3.714722
171,134717,L. Trossard,14,14,9,0,1251,1251,7,2,...,22,46,32,141,101,Midfielders,71,80,92.605728,-21.605728
294,302292,M. Ødegaard,13,13,10,0,1133,1133,6,2,...,20,33,19,149,131,Midfielders,64,77,63.598347,0.401653
273,284302,J. Maddison,13,13,2,0,1185,1185,7,2,...,37,52,26,182,166,Midfielders,82,77,79.715172,2.284828
415,520291,B. Saka,14,14,8,0,1247,1247,4,5,...,31,80,50,137,117,Midfielders,80,76,73.726479,6.273521
364,447205,P. Foden,14,11,6,3,999,999,7,2,...,24,36,18,143,124,Midfielders,83,76,88.744797,-5.744797
65,578325,W. Saliba,14,14,1,0,1353,1353,2,0,...,49,12,7,463,455,Defenders,53,74,46.841492,6.158508


In [21]:
df[df.name == "W. Zaha"]

,playerId,name,matches,matchesInStart,matchesSubstituted,matchesComingOff,minutesOnField,minutesTagged,goals,assists,...,newDefensiveDuelsWon,newOffensiveDuelsWon,newSuccessfulDribbles,lateralPasses,successfulLateralPasses,position,cost,points,predictedCost,predictedCostDiff
150,8422,W. Zaha,13,13,2,0,1249,1249,6,1,...,50,120,63,87,75,Midfielders,75,64,66.353256,8.646744
